In [ ]:
import numpy as np
import numpy.matlib
import torch
import math
import pdb
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn.functional as F

In [ ]:
e_path = '/home/sci/amanpreet/Documents/DT_KG/Codes/Registered/Testing/ellipse.tif'
c_path = '/home/sci/amanpreet/Documents/DT_KG/Codes/Registered/Testing/circle.tif'

ellipse = np.asarray(Image.open(e_path))
circle = np.asarray(Image.open(c_path))

print(circle.shape)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(circle)
ax2.imshow(ellipse)
plt.show()

In [ ]:
print(np.max(circle))

In [ ]:
print(np.array([1 ,0 ,-1]))

In [ ]:
def grad(f):
    f = f.view(1,1,f.shape[0],f.shape[1])
    x_filter = torch.from_numpy(np.reshape(np.array([[0,0,0],[1 ,0 ,-1],[0,0,0]]),[1,1,3,3])).type(torch.FloatTensor).cuda()
    y_filter = torch.from_numpy(np.reshape(np.array([[0,1,0],[0 ,0 ,0],[0,-1,0]]),[1,1,3,3])).type(torch.FloatTensor).cuda()
    ux = F.conv2d(f,x_filter,padding=1)
    uy = F.conv2d(f,y_filter,padding=1)
    return ux[0,0,:,:], uy[0,0,:,:]

In [ ]:
def interp2(f,hy,hx):
    f = f.view(1,1,f.shape[0],f.shape[1])
       
    grid = torch.stack([hy,hx], dim=2)
    grid = grid.view(1,grid.shape[0],grid.shape[1],2).type(torch.cuda.FloatTensor)

    new_im = F.grid_sample(f,grid)
    return new_im[0,0,:,:]

In [ ]:
def elastic_deformation(i1, i2, sigma, l, n_iter, step_size):
    lookup_field_y = np.matlib.repmat(np.arange(i1.shape[1]),i1.shape[0],1)
    lookup_field_x = np.transpose(np.matlib.repmat(np.arange(i1.shape[0]),i1.shape[1],1))
    
    def_field_x = np.zeros(lookup_field_x.shape)
    def_field_y = np.zeros(lookup_field_y.shape)
    
    [r,c] = i1.shape
    L = np.zeros(lookup_field_x.shape)
    
    #Set up the Laplacian for onv Laplacian operation
    for u in range(r):
        for v in range(c):
            L[u][v] = -2.0*(math.cos(2*math.pi*u/r)+math.cos(2*math.pi*v/c))+4.0
            
    #Send everything to torch Tensors and then to GPU
    
    i1 = torch.from_numpy(i1/255.0).type(torch.cuda.FloatTensor)
    i2 = torch.from_numpy(i2/255.0).type(torch.cuda.FloatTensor)
    lookup_field_x = torch.from_numpy(lookup_field_x).type(torch.cuda.FloatTensor)
    lookup_field_y = torch.from_numpy(lookup_field_y).type(torch.cuda.FloatTensor)
    
    #Different identity lookup field in pytorch
    
    lookup_field_x = (lookup_field_x-lookup_field_x.mean())/lookup_field_x.mean() 
    lookup_field_y = (lookup_field_y-lookup_field_y.mean())/lookup_field_y.mean()
    
    def_field_x = torch.from_numpy(def_field_x).type(torch.cuda.FloatTensor)
    def_field_y = torch.from_numpy(def_field_y).type(torch.cuda.FloatTensor)
    L = torch.from_numpy(L).type(torch.cuda.FloatTensor)
    
    # Compute gradient of i2
    
    [i2_x, i2_y] = grad(i2)
    
    #Start the loop
    
    for iter in range(n_iter):
        hx = lookup_field_x + def_field_x
        hy = lookup_field_y + def_field_y

        i2_u = interp2(i2,hy,hx);

        diff = i2_u-i1    

        #Compute the gradient at the deformation

        grad_i2_x = interp2(i2_x,hy,hx)
        grad_i2_y = interp2(i2_y,hy,hx)

        der_x = diff * grad_i2_y
        der_y = diff * grad_i2_x

        # Now take the der to fourier space and apply inverse laplacian

        der_x_f = torch.rfft(der_x,2,onesided=False)       
        der_x_f[:,:,0] = der_x_f[:,:,0]/(L+0.0001)
        der_x_f[:,:,1] = der_x_f[:,:,1]/(L+0.0001)

        der_y_f = torch.rfft(der_y,2,onesided=False)       
        der_y_f[:,:,0] = der_y_f[:,:,0]/(L+0.0001)
        der_y_f[:,:,1] = der_y_f[:,:,1]/(L+0.0001)

        # Now take it back to spatial domain.

        der_x_f = torch.irfft(der_x_f,2,onesided=False)
        der_y_f = torch.irfft(der_y_f,2,onesided=False)    

        def_field_x += (step_size/(sigma**2 * l))*der_x_f
        def_field_y += (step_size/(sigma**2 * l))*der_y_f
        
        if iter%10 == 0:
            f, (ax1, ax2, ax3) = plt.subplots(1, 3)
            ax2.imshow(i2_u.cpu().numpy(), aspect="auto")
            ax2.set_xticks([])
            ax2.set_yticks([])
            ax3.imshow(diff.cpu().numpy(), aspect="auto")
            ax3.set_xticks([])
            ax3.set_yticks([])
            ax1.set_xlim(-1,1)
            ax1.set_ylim(-1,1)
            ax1.plot(hy.cpu().numpy()[0::10,0::10],hx.cpu().numpy()[0::10,0::10],'b')
            ax1.plot(np.transpose(hy.cpu().numpy()[0::10,0::10]),np.transpose(hx.cpu().numpy()[0::10,0::10]),'b')
            ax1.set_xticks([])
            ax1.set_yticks([])
            ax1.invert_yaxis()
            f.set_size_inches(100, 10)
            plt.show()
            print("Difference is", np.sum(diff.cpu().numpy()))
    
    hx = lookup_field_x + def_field_x
    hy = lookup_field_y + def_field_y
    return [interp2(i2,hy,hx).cpu().numpy(), hx.cpu().numpy(), hy.cpu().numpy()]


In [ ]:
[a,b,c] = elastic_deformation(circle,ellipse,500,1,100,100)

In [ ]:
im1_path = '/home/sci/amanpreet/Documents/DT_KG/Codes/Registered/828_24.tif'
im2_path = '/home/sci/amanpreet/Documents/DT_KG/Codes/Registered/829_22.tif'
im1 = np.asarray(Image.open(im1_path))
im2 = np.asarray(Image.open(im2_path))

In [ ]:
[reg_img, hx, hy] = elastic_deformation(im1,im2,500,1,200,200)

In [ ]:
def diffeomorphic_deformation(i1, i2, sigma, l, n_iter, step_size):
    lookup_field_y = np.matlib.repmat(np.arange(i1.shape[1]),i1.shape[0],1)
    lookup_field_x = np.transpose(np.matlib.repmat(np.arange(i1.shape[0]),i1.shape[1],1))
    
    def_field_x = np.zeros(lookup_field_x.shape)
    def_field_y = np.zeros(lookup_field_y.shape)
    
    [r,c] = i1.shape
    L = np.zeros(lookup_field_x.shape)
    
    #Set up the Laplacian for onv Laplacian operation
    for u in range(r):
        for v in range(c):
            L[u][v] = -2.0*(math.cos(2*math.pi*u/r)+math.cos(2*math.pi*v/c))+4.0
            
    #Send everything to torch Tensors and then to GPU
    
    i1 = torch.from_numpy(i1/255.0).type(torch.cuda.FloatTensor)
    i2 = torch.from_numpy(i2/255.0).type(torch.cuda.FloatTensor)
    lookup_field_x = torch.from_numpy(lookup_field_x).type(torch.cuda.FloatTensor)
    lookup_field_y = torch.from_numpy(lookup_field_y).type(torch.cuda.FloatTensor)
    
    #Different identity lookup field in pytorch
    
    lookup_field_x = (lookup_field_x-lookup_field_x.mean())/lookup_field_x.mean() 
    lookup_field_y = (lookup_field_y-lookup_field_y.mean())/lookup_field_y.mean()
    
    def_field_x = torch.from_numpy(def_field_x).type(torch.cuda.FloatTensor)
    def_field_y = torch.from_numpy(def_field_y).type(torch.cuda.FloatTensor)
    L = torch.from_numpy(L).type(torch.cuda.FloatTensor)
    
    # Compute gradient of i2
    
    [i2_x, i2_y] = grad(i2)
    
    #Start the loop
    
    for iter in range(n_iter):
        hx = lookup_field_x + def_field_x
        hy = lookup_field_y + def_field_y

        i2_u = interp2(i2,hy,hx);

        diff = i2_u-i1    

        #Compute the gradient at the deformation

        grad_i2_x = interp2(i2_x,hy,hx)
        grad_i2_y = interp2(i2_y,hy,hx)

        der_x = diff * grad_i2_y
        der_y = diff * grad_i2_x

        # Now take the der to fourier space and apply inverse laplacian

        der_x_f = torch.rfft(der_x,2,onesided=False)       
        der_x_f[:,:,0] = der_x_f[:,:,0]/(L+0.0001)
        der_x_f[:,:,1] = der_x_f[:,:,1]/(L+0.0001)

        der_y_f = torch.rfft(der_y,2,onesided=False)       
        der_y_f[:,:,0] = der_y_f[:,:,0]/(L+0.0001)
        der_y_f[:,:,1] = der_y_f[:,:,1]/(L+0.0001)

        # Now take it back to spatial domain.

        der_x_f = torch.irfft(der_x_f,2,onesided=False)
        der_y_f = torch.irfft(der_y_f,2,onesided=False)    

        velocity_f_x = (step_size/(sigma**2 * l))*der_x_f
        velocity_f_y = (step_size/(sigma**2 * l))*der_y_f
        
        cx = lookup_field_x + velocity_f_x
        cy = lookup_field_y + velocity_f_y
        
        cx[cx > 1] = 1
        cx[cx < -1] = -1
        
        cy[cy > 1] = 1
        cy[cy < -1] = -1
        
        def_field_x = interp2(hx, cy, cx) - lookup_field_x
        def_field_y = interp2(hy, cy, cx) - lookup_field_y        
        
        if iter%10 == 0:
            f, (ax1, ax2, ax3) = plt.subplots(1, 3)
            ax2.imshow(i2_u.cpu().numpy(), aspect="auto")
            ax2.set_xticks([])
            ax2.set_yticks([])
            ax3.imshow(diff.cpu().numpy(), aspect="auto")
            ax3.set_xticks([])
            ax3.set_yticks([])
            ax1.set_xlim(-1,1)
            ax1.set_ylim(-1,1)
            ax1.plot(hy.cpu().numpy()[0::10,0::10],hx.cpu().numpy()[0::10,0::10],'b')
            ax1.plot(np.transpose(hy.cpu().numpy()[0::10,0::10]),np.transpose(hx.cpu().numpy()[0::10,0::10]),'b')
            ax1.set_xticks([])
            ax1.set_yticks([])
            ax1.invert_yaxis()
            f.set_size_inches(100, 10)
            plt.show()
            print("Difference is", np.sum(diff.cpu().numpy()))
    
    hx = lookup_field_x + def_field_x
    hy = lookup_field_y + def_field_y
    return [interp2(i2,hy,hx).cpu().numpy(), hx.cpu().numpy(), hy.cpu().numpy()]

In [ ]:
[a,b,c] = diffeomorphic_deformation(circle,ellipse,1000,1,100,100)